In [1]:
#V21 changes:
    # section #2 - added AttentionPooling
    # section #3 - updated model.classifier added model.vit.pooler 
    # section #15 - updated -> attention_lr, optimizer_grouped_parameters
        # diagnostics printouts and assertions
        # replaced model.parameters with model.vit.parameters

In [2]:
# --------------------------
# 0. Imports
# --------------------------
# Standard Library Imports
import datasets
import csv
import gc
import glob
import multiprocessing as mp
import os
import random
import re
import shutil
import subprocess
import sys
import time

# Third-Party Imports
import accelerate
import dill
import face_recognition
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
#import tensorflow as tf
import torch
import torch.nn.functional as F
import torchvision.transforms as T
import transformers

# From Imports
from collections import Counter
from datasets import ClassLabel, Dataset, Features, Image as DatasetsImage, concatenate_datasets, load_dataset
from datetime import datetime
from functools import partial
from imagehash import phash, hex_to_hash
from io import BytesIO
from pathlib import Path
from PIL import Image, ImageOps, ExifTags, UnidentifiedImageError
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, log_loss, precision_recall_fscore_support
from torch import nn
from torch.nn import functional as F
from torch.optim import AdamW, LBFGS
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torchvision import transforms
from torchvision.transforms import (
    GaussianBlur,
    RandAugment,
    RandomAffine,
    RandomApply,
    RandomPerspective,
    RandomAdjustSharpness,
    ToPILImage,
    ToTensor
)
from tqdm import tqdm
from transformers import (
    AutoImageProcessor,
    AutoModelForImageClassification,
    EarlyStoppingCallback,
    TrainingArguments,
    Trainer,
)

In [3]:
# --------------------------
# 1. Global Configurations
# --------------------------
RUN_INFERENCE = True  # Toggle this off to disable running inference
IMAGE_DIR = "/Users/natalyagrokh/AI/ml_expressions/img_datasets/ferckjalfaga_dataset"
BASE_PATH = IMAGE_DIR

LABEL_NAMES = [
    'anger', 'disgust', 'fear', 'happiness', 'neutral',
    'questioning', 'sadness', 'surprise', 'contempt', 'unknown'
]
id2label = dict(enumerate(LABEL_NAMES))
label2id = {v: k for k, v in id2label.items()}

HARD_CLASS_NAMES = ['contempt', 'disgust', 'fear', 'questioning']
hard_class_ids = [label2id[n] for n in HARD_CLASS_NAMES]

VALID_EXTENSIONS = (".jpg", ".jpeg", ".png", ".tif", ".tiff")

def is_valid_image(filename):
    return filename.lower().endswith(VALID_EXTENSIONS) and not filename.startswith("._")

label_mapping = {name.lower(): name for name in LABEL_NAMES}

# 🔢 Dynamically determine the next version
def get_next_version(base_dir):

    # Use glob to find all entries matching the pattern
    all_entries = glob.glob(os.path.join(base_dir, "V*_*"))
    
    # Filter to include only directories
    existing = [
        os.path.basename(d) for d in all_entries if os.path.isdir(d)
    ]

    # Extract version numbers from the directory names
    versions = [
        int(d[1:].split("_")[0]) for d in existing
        if d.startswith("V") and "_" in d and d[1:].split("_")[0].isdigit()
    ]
    
    # Determine the next version number
    next_version = max(versions, default=0) + 1
    return f"V{next_version}"

# Automatically create a versioned output folder
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
VERSION = get_next_version("/Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training")
VERSION_TAG = VERSION + "_" + timestamp
SAVE_DIR = os.path.join("/Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training", VERSION_TAG)
LOGITS_PATH = os.path.join(SAVE_DIR, f"logits_eval_{VERSION}.npy")
LABELS_PATH = os.path.join(SAVE_DIR, f"labels_eval_{VERSION}.npy")
os.makedirs(SAVE_DIR, exist_ok=True)
print(f"📁 Output directory created: {SAVE_DIR}")

📁 Output directory created: /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V21_20250630_142520


In [29]:
# --------------------------
# 2. Utility Functions (Metrics & Calibration)
# --------------------------

# performs attention over patch tokens to create single context vector -
    # more robust way to implement attention mechanism   
class AttentionPooling(nn.Module):
    
    def __init__(self, hidden_size):
        super().__init__()
        self.attention_net = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.Tanh(),
            nn.Linear(hidden_size // 2, 1)
        )

    def forward(self, hidden_states):
        # hidden_states shape: (batch_size, sequence_length, hidden_size)
        # We only use the patch tokens for attention, ignoring the [CLS] token
        patch_tokens = hidden_states[:, 1:, :]
        
        # Compute attention scores
        # Shape: (batch_size, sequence_length - 1, 1)
        attention_scores = self.attention_net(patch_tokens)
        
        # Apply softmax to get attention weights
        # Shape: (batch_size, sequence_length - 1, 1)
        attention_weights = F.softmax(attention_scores, dim=1)
        
        # Compute the context vector (weighted sum of patch tokens)
        # Shape: (batch_size, hidden_size)
        context_vector = torch.sum(attention_weights * patch_tokens, dim=1)
        
        # The ViT pooler is expected to return a pooled output of this shape
        return context_vector


# 🔍 Compute perceptual hash for image similarity clustering (used in REVIEW and Disgust curation)
def compute_hash(image_path):
    try:
        img = Image.open(image_path).convert("L").resize((64, 64))
        return str(phash(img))
    except Exception:
        return None


# Inject image_path BEFORE any map/filter
def add_image_path(example):
    # Handle DatasetsImage and PIL.Image types robustly
    img_obj = example["image"]
    path = getattr(img_obj, "filename", None)
    if path is None:
        # Fallback: Try reconstructing path from folder and file (rarely needed)
        if "file" in example:
            path = os.path.join(BASE_PATH, example["file"])
        else:
            path = ""
    example["image_path"] = path
    return example


# Reconcile labels as before, but preserve all fields
def reconcile_labels(example):
    label = example.get("label", None)
    if isinstance(label, int):
        original_label = dataset.features["label"].int2str(label).strip().lower()
    elif isinstance(label, str):
        original_label = label.strip().lower()
    else:
        file_path = example["image_path"]
        original_label = os.path.basename(os.path.dirname(file_path)).lower() if file_path else None
    pretrain_label = label_mapping.get(original_label)
    example["label"] = label2id[pretrain_label] if pretrain_label is not None else -1
    return example    


# Define custom Trainer to inject class weights
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        # Use smoothed CE + confidence penalty
        smooth_ce_loss = SmoothedCrossEntropyLoss(smoothing=0.05)
        loss = smooth_ce_loss(logits, labels) + confidence_penalty(logits, beta=0.05)
        return (loss, outputs) if return_outputs else loss


def modified_train(*args, **kwargs):
    result = original_train(*args, **kwargs)
    scheduler.step(trainer.state.epoch)  # instead of eval_loss
    return result
    

# 🔄 Smoothed Cross Entropy Loss (ε = 0.05)
class SmoothedCrossEntropyLoss(nn.Module):
    def __init__(self, smoothing=0.05):
        super().__init__()
        self.smoothing = smoothing

    def forward(self, logits, target):
        num_classes = logits.size(1)
        with torch.no_grad():
            smooth_labels = torch.full_like(logits, self.smoothing / (num_classes - 1))
            smooth_labels.scatter_(1, target.unsqueeze(1), 1.0 - self.smoothing)
        log_probs = F.log_softmax(logits, dim=1)
        loss = -(smooth_labels * log_probs).sum(dim=1).mean()
        return loss


# ⚠️ Confidence Penalty to Reduce Overconfidence
def confidence_penalty(logits, beta=0.05):
    probs = F.softmax(logits, dim=1)
    log_probs = F.log_softmax(logits, dim=1)
    entropy = -torch.sum(probs * log_probs, dim=1)
    return beta * entropy.mean()


# 📊 Compute Metrics with Confusion Matrix Logging
def compute_metrics_with_confusion(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    # Print classification report
    print("\nClassification Report:")
    report = classification_report(labels, preds, target_names=LABEL_NAMES, output_dict=True)
    print(classification_report(labels, preds, target_names=LABEL_NAMES))

    # Save raw logits/labels for calibration or further analysis
    np.save(os.path.join(SAVE_DIR, f"logits_eval_{VERSION}.npy"), logits)
    np.save(os.path.join(SAVE_DIR, f"labels_eval_{VERSION}.npy"), labels)

    # Save per-class F1/precision/recall/entropy to CSV (append per epoch)
    f1s = [report[name]["f1-score"] for name in LABEL_NAMES]
    recalls = [report[name]["recall"] for name in LABEL_NAMES]
    precisions = [report[name]["precision"] for name in LABEL_NAMES]

    # Entropy per class (sorted by entropy)
    softmax_probs = F.softmax(torch.tensor(logits), dim=-1)
    entropies = -torch.sum(softmax_probs * torch.log(softmax_probs + 1e-12), dim=-1)
    entropy_per_class = []
    for idx, class_name in enumerate(LABEL_NAMES):
        mask = (np.array(labels) == idx)
        if mask.any():
            class_entropy = entropies[mask].mean().item()
            entropy_per_class.append((class_name, class_entropy))
        else:
            entropy_per_class.append((class_name, 0.0))
    # Sort for display only; CSV row stays in canonical label order
    sorted_entropy = sorted(entropy_per_class, key=lambda x: x[1], reverse=True)

    # CSV logging
    epoch_metrics_path = os.path.join(SAVE_DIR, "per_class_metrics.csv")
    epoch = getattr(trainer.state, "epoch", None) if "trainer" in globals() else None
    df_row = pd.DataFrame({
        "epoch": [epoch],
        **{f"f1_{n}": [f] for n, f in zip(LABEL_NAMES, f1s)},
        **{f"recall_{n}": [r] for n, r in zip(LABEL_NAMES, recalls)},
        **{f"precision_{n}": [p] for n, p in zip(LABEL_NAMES, precisions)},
        **{f"entropy_{n}": [e] for n, e in entropy_per_class}
    })
    if os.path.exists(epoch_metrics_path):
        df_row.to_csv(epoch_metrics_path, mode="a", header=False, index=False)
    else:
        df_row.to_csv(epoch_metrics_path, mode="w", header=True, index=False)

    # Generate and print confusion matrix heatmap
    cm = confusion_matrix(labels, preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(
        cm, annot=True, fmt="d", cmap="Blues",
        xticklabels=LABEL_NAMES,
        yticklabels=LABEL_NAMES
    )
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Confusion Matrix")
    plt.tight_layout()
    plt.savefig(os.path.join(SAVE_DIR, f"confusion_matrix_epoch_{VERSION}.png"))
    plt.close()

    # Top confused pairs
    confusion_pairs = [
        ((LABEL_NAMES[i], LABEL_NAMES[j]), cm[i][j])
        for i in range(len(LABEL_NAMES))
        for j in range(len(LABEL_NAMES)) if i != j
    ]
    top_confusions = sorted(confusion_pairs, key=lambda x: x[1], reverse=True)[:3]
    print("\nTop 3 confused class pairs:")
    for (true_label, pred_label), count in top_confusions:
        print(f"  - {true_label} → {pred_label}: {count} instances")

    # Compute average prediction entropy
    avg_entropy = entropies.mean().item()
    print(f"\n🧠 Avg prediction entropy: {avg_entropy:.4f}")

    print("\n🔍 Class entropies (sorted):")
    for class_name, entropy in sorted_entropy:
        print(f"  - {class_name}: entropy = {entropy:.4f}")

    accuracy = (preds == labels).mean()
    return {"accuracy": accuracy}


# 🌡️ Apply Temperature Scaling for Calibration
def apply_temperature_scaling(logits_path, labels_path):
    if not (os.path.exists(logits_path) and os.path.exists(labels_path)):
        print(f"❌ Missing files:\n  - {logits_path if not os.path.exists(logits_path) else ''}\n - {labels_path if not os.path.exists(labels_path) else ''}")
        return None

    print(f"📂 Loading logits from: {logits_path}")
    print(f"📂 Loading labels from: {labels_path}")

    logits = torch.tensor(np.load(logits_path), dtype=torch.float32).to(device)
    labels = torch.tensor(np.load(labels_path), dtype=torch.long).to(device)

    class TemperatureScaler(nn.Module):
        def __init__(self):
            super().__init__()
            self.temperature = nn.Parameter(torch.ones(1) * 1.5)

        def forward(self, logits):
            return logits / self.temperature

    model = TemperatureScaler().to(device)
    optimizer = LBFGS([model.temperature], lr=0.01, max_iter=50)

    def eval_fn():
        optimizer.zero_grad()
        loss = F.cross_entropy(model(logits), labels)
        loss.backward()
        return loss

    optimizer.step(eval_fn)
    calibrated_logits = model(logits)
    probs = F.softmax(calibrated_logits, dim=1).detach().cpu().numpy()
    logloss = log_loss(labels.cpu().numpy(), probs)

    # Save optimal temperature
    temperature_value = model.temperature.item()
    torch.save(
        torch.tensor([temperature_value]),
        os.path.join(SAVE_DIR, f"{VERSION}_calibrated_temperature.pt")
    )
    print(f"✅ Optimal temperature: {temperature_value:.4f}")
    print(f"✅ Calibrated Log Loss: {logloss:.4f}")
    return temperature_value, logits.cpu(), labels.cpu()


# 📈 Plot Reliability Diagram (Calibration Curve)
def plot_reliability_diagram(logits, labels, temperature, n_bins=15):
    probs = F.softmax(logits / temperature, dim=1)
    confidences, predictions = torch.max(probs, 1)
    accuracies = predictions.eq(labels)

    bins = torch.linspace(0, 1, n_bins + 1)
    bin_lowers, bin_uppers = bins[:-1], bins[1:]

    bin_accuracies, bin_confidences = [], []
    for lower, upper in zip(bin_lowers, bin_uppers):
        mask = (confidences > lower) & (confidences <= upper)
        if mask.any():
            bin_accuracies.append(accuracies[mask].float().mean())
            bin_confidences.append(confidences[mask].mean())

    plt.figure(figsize=(6, 6))
    plt.plot(bin_confidences, bin_accuracies, marker='o', label='Model')
    plt.plot([0, 1], [0, 1], linestyle='--', label='Perfect Calibration')
    plt.title("Reliability Diagram (After Temperature Scaling)")
    plt.xlabel("Confidence")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    output_path = os.path.join(SAVE_DIR, f"{VERSION}_reliability_diagram_calibrated.png")
    plt.savefig(output_path)
    plt.close()
    print(f"📊 Saved reliability diagram to {output_path}")

# saving model and processor
def save_model_and_processor(model, processor, save_dir, trainer=None):
    print(f"Saving model and processor to: {save_dir}")
    
    model = model.to("cpu")

    # Save processor
    processor.save_pretrained(save_dir)
    print(f"✅ Processor saved to: {SAVE_DIR}")
    
    # Save full model
    model.save_pretrained(SAVE_DIR, safe_serialization=True)
    print(f"✅ Full model saved to: {SAVE_DIR}")

    # Save state dict
    final_model_path = os.path.join(SAVE_DIR, 'final_model.pth')
    torch.save(model.state_dict(), final_model_path)
    print(f"✅ State dict saved to: {final_model_path}")

    # Save trainer state
    if trainer is not None:
        try:
            trainer.save_model(os.path.join(save_dir, "backup_trainer_model"))
            print("✅ Trainer backup saved.")
        except Exception as e:
            print(f"⚠️ Failed to save trainer backup: {e}")

    # Memory cleanup
    del model
    gc.collect()
    try:
        torch.cuda.empty_cache()
    except Exception:
        pass  # Not all systems have CUDA
    print("✅ Memory cleanup complete after save.")


# 🚦 Prints label distribution for a dataset
    #only calling for ad hoc debugging, experiments, sanity checks 
def check_label_integrity(dataset, LABEL_NAMES, label2id):
    # Count all mapped labels
    label_counts = Counter(dataset['label'])
    print("\n🚨 Label distribution after mapping (before split):")
    for label_id in range(len(LABEL_NAMES)):
        label_name = LABEL_NAMES[label_id]
        print(f"  {label_name:12}: {label_counts.get(label_id, 0)}")

    # Specifically highlight 'surprise'
    surprise_id = label2id['surprise']
    if label_counts.get(surprise_id, 0) == 0:
        print("❗WARNING: No 'surprise' images found after mapping!")
    elif label_counts[surprise_id] < 50:  # arbitrary threshold
        print(f"⚠️ Only {label_counts[surprise_id]} 'surprise' images found! Check curation or mapping.")


# 🚦 Prints label distribution for multiple datasets
def check_all_label_integrity(datasets_dict, LABEL_NAMES, label2id):
    for name, dataset in datasets_dict.items():
        print(f"\n🚨 Label distribution for: {name}")
        label_counts = Counter(dataset['label'])
        for label_id in range(len(LABEL_NAMES)):
            label_name = LABEL_NAMES[label_id]
            print(f"  {label_name:12}: {label_counts.get(label_id, 0)}")
        surprise_id = label2id['surprise']
        if label_counts.get(surprise_id, 0) == 0:
            print("❗WARNING: No 'surprise' images found in this split!")
        elif label_counts[surprise_id] < 50:
            print(f"⚠️ Only {label_counts[surprise_id]} 'surprise' images in {name}! Check curation or mapping.")


# --- Stronger Augmentation Utility ---
def make_transform_function(processor, hard_class_ids):
    def transform_function(example):
        label = example["label"]
        aug_pipeline = strong_aug if label in hard_class_ids else data_augment
        if example["image"].mode != "RGB":
            example["image"] = example["image"].convert("RGB")
        augmented_image = aug_pipeline(example["image"])
        inputs = processor(augmented_image, return_tensors="pt")
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        inputs["labels"] = example["label"]
        return inputs
    return transform_function  

In [5]:
# --------------------------
# 3. Auto-Load Latest Pretrained Model and Processor
# --------------------------

MODEL_ROOT = "/Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training"

def extract_version(dirname):
    match = re.match(r"V(\d+)", os.path.basename(dirname))
    return int(match.group(1)) if match else -1

model_dirs = [
    os.path.join(MODEL_ROOT, d)
    for d in os.listdir(MODEL_ROOT)
    if d.startswith("V") and os.path.isdir(os.path.join(MODEL_ROOT, d))
]

model_dirs = [d for d in model_dirs if os.path.abspath(d) != os.path.abspath(SAVE_DIR)]
model_dirs = sorted(model_dirs, key=extract_version, reverse=True)

if not model_dirs:
    raise FileNotFoundError("❌ No earlier model folders found.")

# We still load from the latest V20 fine-tuned model
model_path = model_dirs[0]
print(f"✅ Auto-loading base ViT from: {model_path}")

# Load the processor (this doesn't change)
processor = AutoImageProcessor.from_pretrained(model_path)

# Load the standard ViT model for image classification
model = AutoModelForImageClassification.from_pretrained(model_path)

# --- SURGICAL REPLACEMENT ---
# Replace the model's default pooler with our new AttentionPooling layer.
# This is a much more stable way to introduce the attention mechanism.
print("⚠️ Replacing default ViT pooler with custom AttentionPooling layer.")
model.vit.pooler = AttentionPooling(model.config.hidden_size)
print("✅ Pooler replaced successfully.")

# Replace classification head to match current label schema
model.classifier = nn.Linear(model.config.hidden_size, len(LABEL_NAMES))
model.config.id2label = id2label
model.config.label2id = label2id
model.config.num_labels = len(LABEL_NAMES)
print("✅ Classifier head reset for new training.")


# Define device and push model to device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("\n🖥️ Using device:", device)
model.to(device).eval()

✅ Auto-loading base ViT from: /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V20_20250629_134956
⚠️ Replacing default ViT pooler with custom AttentionPooling layer.
✅ Pooler replaced successfully.
✅ Classifier head reset for new training.

🖥️ Using device: mps


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (intermed

In [6]:
# ==============================
# 4. Load and Prepare Dataset (with filename preservation)
# ==============================

# --- Dynamic File Count ---
print("🔍 Counting valid image files on disk for verification...")
# This will recursively find all valid image files in your dataset directory
expected_file_count = len(
    [p for p in Path(BASE_PATH).rglob("*") if is_valid_image(p.name)]
)
print(f"✅ Found {expected_file_count} image files in {BASE_PATH}")

# Disable caching BEFORE loading
datasets.disable_caching()
print("✅ Datasets caching disabled for this run to ensure fresh data load.")

# Step 1: Load dataset and capture filepaths
dataset = load_dataset(
    "imagefolder",
    data_dir=BASE_PATH,
    split="train" # No need to specify cache_dir when caching is off
)

# Only run ONCE and only here, so "image_path" is never dropped later!
dataset = dataset.map(add_image_path, desc="Add file path to each record")
dataset = dataset.map(reconcile_labels, desc="Re-labeling dataset (preserving image_path)")
dataset = dataset.filter(lambda x: x["label"] != -1)

# ** Robust Verification **
final_count = len(dataset)
print(f"✅ Total examples after filtering: {final_count}")
print("Sample with path:", dataset[0]["image_path"])

# Assertion checks whether loaded count is very close to the disk count
# Small tolerance accounts for any files that fail to load or be filtered
assert abs(final_count - expected_file_count) < 10, \
    f"Dataset size mismatch! Found {expected_file_count} files but loaded {final_count}."

assert dataset[0].get("image_path", None), "image_path missing from first record"

🔍 Counting valid image files on disk for verification...
✅ Found 17501 image files in /Users/natalyagrokh/AI/ml_expressions/img_datasets/ferckjalfaga_dataset
✅ Datasets caching disabled for this run to ensure fresh data load.


Resolving data files:   0%|          | 0/17510 [00:00<?, ?it/s]

Add file path to each record:   0%|          | 0/17500 [00:00<?, ? examples/s]

Re-labeling dataset (preserving image_path):   0%|          | 0/17500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/17500 [00:00<?, ? examples/s]

✅ Total examples after filtering: 17500
Sample with path: /Users/natalyagrokh/AI/ml_expressions/img_datasets/ferckjalfaga_dataset/anger/Abel_Pacheco_0002.jpg


In [7]:
# --------------------------
# 5. Dataset Label Overview and Folder Stats
# --------------------------
def analyze_dataset_structure(dataset, id2label, base_path):
    # Print label schema from the dataset
    print("Label schema (from dataset):", dataset.features["label"])

    # Label distribution from the dataset object
    label_counts = Counter(dataset["label"])
    print("\n📊 Full dataset label distribution (from Dataset object):")
    for label_id, count in sorted(label_counts.items()):
        print(f"  {id2label[label_id]}: {count} examples")

    # Dynamically detect minority classes (lowest 3 frequencies)
    N = 3
    minority_classes = set(
        label for label, _ in sorted(label_counts.items(), key=lambda x: x[1])[:N]
    )
    print(f"\n⚠️  Dynamically identified minority classes: {[id2label[i] for i in minority_classes]}")

    # Count images per directory, and store for later validation
    folder_image_counts = {}
    print("\n📂 Image count per label folder:")
    for label in sorted(os.listdir(base_path)):
        label_path = os.path.join(base_path, label)
        if os.path.isdir(label_path):
            valid_images = [img for img in os.listdir(label_path) if is_valid_image(img)]
            folder_image_counts[label] = len(valid_images)
            print(f"  {label}: {len(valid_images)} images")

    return minority_classes, folder_image_counts

# Example usage right after dataset loading
minority_classes, folder_image_counts = analyze_dataset_structure(dataset, id2label, BASE_PATH)

Label schema (from dataset): ClassLabel(names=['anger', 'contempt', 'disgust', 'fear', 'happiness', 'neutral', 'questioning', 'sadness', 'surprise', 'unknown'], id=None)

📊 Full dataset label distribution (from Dataset object):
  anger: 2302 examples
  disgust: 309 examples
  fear: 1432 examples
  happiness: 2892 examples
  neutral: 3334 examples
  questioning: 1939 examples
  sadness: 1706 examples
  surprise: 2779 examples
  contempt: 421 examples
  unknown: 386 examples

⚠️  Dynamically identified minority classes: ['contempt', 'disgust', 'unknown']

📂 Image count per label folder:
  anger: 2302 images
  contempt: 421 images
  disgust: 309 images
  fear: 1432 images
  happiness: 2892 images
  neutral: 3334 images
  questioning: 1939 images
  sadness: 1706 images
  surprise: 2779 images
  unknown: 386 images


In [9]:
# --------------------------
# 6. Perceptual Clustering for Ambiguous/Confused Classes
# --------------------------

CLUSTER_TARGETS = ["disgust", "sadness", "fear", "questioning", "contempt"]

for class_name in CLUSTER_TARGETS:
    class_dir = os.path.join(BASE_PATH, class_name)
    if not os.path.isdir(class_dir):
        print(f"⚠️ Class dir not found: {class_dir} (skipping)")
        continue

    class_images = [
        os.path.join(class_dir, f) for f in os.listdir(class_dir)
        if is_valid_image(f)
    ]
    hash_map = {}
    for path in class_images:
        h = compute_hash(path)
        if h:
            hash_map.setdefault(h, []).append(path)

    cluster_dir = os.path.join(SAVE_DIR, f"{class_name}_clusters")
    os.makedirs(cluster_dir, exist_ok=True)

    print(f"🔍 {class_name.capitalize()} hash clusters with more than 1 image:")
    for h, paths in hash_map.items():
        if len(paths) > 1:
            cluster_path = os.path.join(cluster_dir, h)
            os.makedirs(cluster_path, exist_ok=True)
            for p in paths:
                shutil.copy(p, cluster_path)
            print(f"  - Cluster {h[:8]}: {len(paths)} images copied for review")

🔍 Disgust hash clusters with more than 1 image:
🔍 Sadness hash clusters with more than 1 image:
  - Cluster 958c52e1: 2 images copied for review
  - Cluster ee9a8d33: 2 images copied for review
  - Cluster d0890396: 2 images copied for review
  - Cluster bb0d06f2: 2 images copied for review
  - Cluster d7f00fa2: 2 images copied for review
🔍 Fear hash clusters with more than 1 image:
  - Cluster 9ae56592: 2 images copied for review
  - Cluster 91c8ee81: 2 images copied for review
  - Cluster dae5a596: 2 images copied for review
🔍 Questioning hash clusters with more than 1 image:
  - Cluster da014886: 2 images copied for review
  - Cluster 9db42783: 2 images copied for review
🔍 Contempt hash clusters with more than 1 image:


In [10]:
# --------------------------
# 7. Class Frequency-Aware Augmentation Targeting
# --------------------------

# Compute label frequencies from train split (post filtering)
label_freqs = Counter(dataset["label"])
label_id2name = {v: k for k, v in label2id.items()}
label_name2id = {v: k for k, v in label_id2name.items()}

# Get lowest-count classes dynamically
minority_by_count = sorted(label_freqs, key=label_freqs.get)[:3]
minority_by_name = [label_id2name[i] for i in minority_by_count]
minority_by_name = [n for n in minority_by_name if n != "unknown"]

# Manually include known confused or underperforming classes
manual_focus_classes = ['disgust', 'questioning', 'contempt']

# Merge and deduplicate
minority_class_names = list(set(minority_by_name + manual_focus_classes))

# Final list as label indices
minority_classes = [label_name2id[name] for name in minority_class_names]

print(f"🎯 Targeted minority augmentation will apply to: {minority_class_names}")

🎯 Targeted minority augmentation will apply to: ['questioning', 'contempt', 'disgust']


In [11]:
# --------------------------
# 8. Define Data Augmentation and Preprocessing Transformation
# --------------------------

# Baseline augmentation
data_augment = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomRotation(10),
    T.ColorJitter(brightness=0.1, contrast=0.1)
])

# RandAugment for specific minority classes only
minority_classes_names = minority_class_names
minority_classes = [label2id[label] for label in minority_classes_names]

minority_aug = T.Compose([
    RandAugment(num_ops=2, magnitude=9),
    T.RandomResizedCrop(224, scale=(0.7, 1.0)),
    T.ColorJitter(0.3, 0.3, 0.3, 0.1),
])

# Augmentation counter tracking
aug_count = Counter()

def make_transform_function(processor, minority_classes):
    def transform_function(example):
        label = example["label"]
        aug_pipeline = minority_aug if label in minority_classes else data_augment
        aug_count[label] += 1

        if example["image"].mode != "RGB":
            example["image"] = example["image"].convert("RGB")

        augmented_image = aug_pipeline(example["image"])
        inputs = processor(augmented_image, return_tensors="pt")
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        inputs["labels"] = example["label"]
        return inputs
    return transform_function

# After mapping finishes:
dataset = dataset.map(make_transform_function(processor, minority_classes))
formatted_counts = {LABEL_NAMES[k]: v for k, v in aug_count.items()}
print(f"✅ Augmentation counts: {formatted_counts}")

# Explicitly log dataset snapshots (class distribution) to a 
# CSV or JSON after each run for easy future diffing and tracking
snapshot_path = os.path.join(SAVE_DIR, f"{VERSION}_augmentation_snapshot.csv")
aug_snapshot = pd.DataFrame.from_dict(dict(aug_count), orient='index', columns=['count'])
aug_snapshot.to_csv(snapshot_path)

print(f"✅ Saved augmentation snapshot to {snapshot_path}")

Map:   0%|          | 0/17500 [00:00<?, ? examples/s]

✅ Augmentation counts: {'anger': 2302, 'contempt': 421, 'disgust': 309, 'fear': 1432, 'happiness': 2892, 'neutral': 3334, 'questioning': 1939, 'sadness': 1706, 'surprise': 2779, 'unknown': 386}
✅ Saved augmentation snapshot to /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V21_20250630_142520/V21_augmentation_snapshot.csv


In [12]:
# --------------------------
# 9. Train-Validation Split
# --------------------------
split_dataset = dataset.train_test_split(test_size=0.2)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

# 🚦 Check and print label distributions across all important splits
check_all_label_integrity(
    {
        "full dataset (post-aug)": dataset,
        "train set": train_dataset,
        "val set": eval_dataset,
        # "post-balance train": train_dataset_balanced,
    },
    LABEL_NAMES, label2id
)


🚨 Label distribution for: full dataset (post-aug)
  anger       : 2302
  disgust     : 309
  fear        : 1432
  happiness   : 2892
  neutral     : 3334
  questioning : 1939
  sadness     : 1706
  surprise    : 2779
  contempt    : 421
  unknown     : 386

🚨 Label distribution for: train set
  anger       : 1838
  disgust     : 244
  fear        : 1157
  happiness   : 2277
  neutral     : 2696
  questioning : 1553
  sadness     : 1390
  surprise    : 2186
  contempt    : 350
  unknown     : 309

🚨 Label distribution for: val set
  anger       : 464
  disgust     : 65
  fear        : 275
  happiness   : 615
  neutral     : 638
  questioning : 386
  sadness     : 316
  surprise    : 593
  contempt    : 71
  unknown     : 77


In [13]:
# --------------------------
# 10. Label Distribution Snapshot and Drift Monitor
# --------------------------
snapshot_dir = os.path.join(SAVE_DIR, "label_snapshots")
os.makedirs(snapshot_dir, exist_ok=True)

# Count current training labels
train_label_names = [LABEL_NAMES[i] for i in train_dataset['label']]
label_counts = pd.Series(train_label_names).value_counts().sort_index()
label_counts.name = VERSION

# Save snapshot CSV
snapshot_path = os.path.join(snapshot_dir, f"{VERSION}_label_distribution.csv")
label_counts.to_csv(snapshot_path)
print(f"📊 Saved label distribution snapshot: {snapshot_path}")

# Optionally compare to previous version
previous_versions = sorted([
    f for f in os.listdir(snapshot_dir) if f.endswith(".csv") and not f.startswith(VERSION)
])
if previous_versions:
    latest_prev = previous_versions[-1]
    prev_df = pd.read_csv(os.path.join(snapshot_dir, latest_prev), index_col=0)
    diff = label_counts.subtract(prev_df.iloc[:, 0], fill_value=0)
    print("🔍 Label count change since last snapshot:")
    print(diff)

# 🚦 Check and print label distributions across all important splits
check_all_label_integrity(
    {
        "full dataset (post-aug)": dataset,
        "train set": train_dataset,
        "val set": eval_dataset,
        # "post-balance train": train_dataset_balanced,
    },
    LABEL_NAMES, label2id
)

📊 Saved label distribution snapshot: /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V21_20250630_142520/label_snapshots/V21_label_distribution.csv

🚨 Label distribution for: full dataset (post-aug)
  anger       : 2302
  disgust     : 309
  fear        : 1432
  happiness   : 2892
  neutral     : 3334
  questioning : 1939
  sadness     : 1706
  surprise    : 2779
  contempt    : 421
  unknown     : 386

🚨 Label distribution for: train set
  anger       : 1838
  disgust     : 244
  fear        : 1157
  happiness   : 2277
  neutral     : 2696
  questioning : 1553
  sadness     : 1390
  surprise    : 2186
  contempt    : 350
  unknown     : 309

🚨 Label distribution for: val set
  anger       : 464
  disgust     : 65
  fear        : 275
  happiness   : 615
  neutral     : 638
  questioning : 386
  sadness     : 316
  surprise    : 593
  contempt    : 71
  unknown     : 77


In [14]:
# --------------------------
# 11. Balance Dataset (with NO oversampling for 'unknown')
# --------------------------
MINORITY_CAP = 2250
balanced_subsets = []
label_counts = Counter(dataset["label"])
print("Original label distribution:", label_counts)

for label, count in label_counts.items():
    subset = dataset.filter(lambda x: x['label'] == label, num_proc=1)
    class_name = LABEL_NAMES[label]
    if class_name == "unknown":
        balanced_subsets.append(subset)
    elif count < MINORITY_CAP:
        multiplier = MINORITY_CAP // len(subset)
        remainder = MINORITY_CAP % len(subset)
        subset = concatenate_datasets([subset] * multiplier + [subset.select(range(remainder))])
        balanced_subsets.append(subset)
    else:
        # Append full set (no downsampling for majority classes)
        balanced_subsets.append(subset)

train_dataset = concatenate_datasets(balanced_subsets).shuffle(seed=42)
print("After balancing:", Counter(train_dataset['label']))

hard_classes = ['contempt', 'disgust', 'questioning', 'surprise', 'fear']
hard_class_ids = [label2id[c] for c in hard_classes]

# Calculate weights: Give hard classes 2x, others 1x
weights = [2.0 if l in hard_class_ids else 1.0 for l in train_dataset["label"]]
weights = torch.DoubleTensor(weights)
sampler = torch.utils.data.WeightedRandomSampler(
    weights=weights,
    num_samples=len(weights),
    replacement=True
)

# 🚦 Check and print label distributions across all important splits
check_all_label_integrity(
    {
        "full dataset (post-aug)": dataset,
        "train set": train_dataset,
        "val set": eval_dataset,
        # "post-balance train": train_dataset_balanced,
    },
    LABEL_NAMES, label2id
)

Original label distribution: Counter({4: 3334, 3: 2892, 7: 2779, 0: 2302, 5: 1939, 6: 1706, 2: 1432, 8: 421, 9: 386, 1: 309})


Filter:   0%|          | 0/17500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/17500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/17500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/17500 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/17500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/17500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/17500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/17500 [00:00<?, ? examples/s]

After balancing: Counter({4: 3334, 3: 2892, 7: 2779, 0: 2302, 2: 2250, 5: 2250, 1: 2250, 6: 2250, 8: 2250, 9: 386})

🚨 Label distribution for: full dataset (post-aug)
  anger       : 2302
  disgust     : 309
  fear        : 1432
  happiness   : 2892
  neutral     : 3334
  questioning : 1939
  sadness     : 1706
  surprise    : 2779
  contempt    : 421
  unknown     : 386

🚨 Label distribution for: train set
  anger       : 2302
  disgust     : 2250
  fear        : 2250
  happiness   : 2892
  neutral     : 3334
  questioning : 2250
  sadness     : 2250
  surprise    : 2779
  contempt    : 2250
  unknown     : 386

🚨 Label distribution for: val set
  anger       : 464
  disgust     : 65
  fear        : 275
  happiness   : 615
  neutral     : 638
  questioning : 386
  sadness     : 316
  surprise    : 593
  contempt    : 71
  unknown     : 77


In [15]:
# ==========================
# 12. Curriculum Learning: Staged Hard/Easy Sample Scheduling (with preserved image_path)
# ==========================

# (A) Use audit_csv_path, or rebuild audit if needed
audit_csv_path = os.path.join(SAVE_DIR, "review_assignment_audit.csv")
if not os.path.exists(audit_csv_path):
    # Rebuild (all records have image_path now!)
    pd.DataFrame([{"image_path": ex["image_path"], "label": ex["label"]}
                  for ex in dataset]).to_csv(audit_csv_path, index=False)

audit_df = pd.read_csv(audit_csv_path)

# (B) Map image_path (full path or filename) to index
dataset_path_to_idx = {
    os.path.basename(ex["image_path"]): i for i, ex in enumerate(dataset)
}

easy_idxs, hard_idxs = [], []
for _, row in audit_df.iterrows():
    path_val = row["image_path"]
    if not isinstance(path_val, str) or not path_val:
        continue
    basename = os.path.basename(path_val)
    idx = dataset_path_to_idx.get(basename)
    if idx is not None:
        # For now, assign all as easy unless you have other columns
        easy_idxs.append(idx)
    else:
        print(f"Image in audit CSV not found in current dataset: {row['image_path']}")

easy_dataset = dataset.select(easy_idxs)
hard_dataset = dataset.select(hard_idxs)
print(f"Curriculum split: {len(easy_dataset)} easy, {len(hard_dataset)} hard")

Curriculum split: 16664 easy, 0 hard


In [16]:
# --------------------------
# 13. Define Training Arguments for Robust Fine-Tuning
# --------------------------
training_args = TrainingArguments(
    output_dir=SAVE_DIR,                   # Directory to save checkpoints and the final model
    eval_strategy="epoch",                 # Evaluate at the end of each epoch
    save_strategy="epoch",                 # Save checkpoint at each epoch
    save_total_limit=2,                    # ✅ (optional) Keep only last 2 checkpoints to save space
    learning_rate=4e-5,                    # A conservative learning rate for fine-tuning
    per_device_train_batch_size=8,         # Adjust based on your CPU memory limits
    per_device_eval_batch_size=8,
    num_train_epochs=5,                    # Fine-tune for a few epochs (adjust as needed)
    load_best_model_at_end=True,           # Automatically load the best model when training finishes
    metric_for_best_model="accuracy",      # Monitor accuracy for best model selection
    logging_dir=os.path.join(SAVE_DIR, "logs"),  # ✅ Save logs inside versioned folder
    logging_strategy="epoch",                 # ✅ Log once per epoch
    save_safetensors=True                  # ✅ Optional: saves model weights in `.safetensors` (safe format)
)

In [17]:
# --------------------------
# 14. Define Compute Metrics
# --------------------------
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = (predictions == labels).mean()
    return {"accuracy": accuracy}

In [18]:
# --------------------------
# 15. Optimizer, Scheduler, and Training (Attention Pooling)
# --------------------------

# --- Part A: Discriminative Learning Rate Optimizer Setup ---
print("--- Setting up optimizer for ViT with Attention Pooling ---")

# Define different learning rates
head_lr = 5e-5      # High learning rate for the new classifier
attention_lr = 5e-5 # High learning rate for our new attention pooling layer
backbone_lr = 2e-7  # Very low learning rate for the fine-tuned backbone layers

# First, freeze all layers of the base ViT model
for param in model.vit.parameters():
    param.requires_grad = False

# Unfreeze the last 2 layers of the ViT encoder for fine-tuning
for name, param in model.vit.encoder.layer[-2:].named_parameters():
    param.requires_grad = True
    print(f"Unfreezing for fine-tuning: {name}")
    
# Unfreeze our new attention pooling layer
for param in model.vit.pooler.parameters():
    param.requires_grad = True
print("Unfreezing for training: vit.pooler (AttentionPooling)")


# Create parameter groups for the optimizer
optimizer_grouped_parameters = [
    {'params': model.classifier.parameters(), 'lr': head_lr},
    {'params': model.vit.pooler.parameters(), 'lr': attention_lr},
    {'params': model.vit.encoder.layer[-2:].parameters(), 'lr': backbone_lr}
]

# Create the AdamW optimizer with the specified parameter groups.
optimizer = torch.optim.AdamW(optimizer_grouped_parameters, weight_decay=0.01)

# --- Part B: Trainer Initialization ---
training_args.load_best_model_at_end = True
training_args.metric_for_best_model = "eval_loss"
training_args.evaluation_strategy = "epoch"
training_args.save_strategy = "epoch"

early_stop_callback = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.001
)

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics_with_confusion,
    optimizers=(optimizer, None),
    callbacks=[early_stop_callback]
)

# --- Part C: Custom Scheduler Setup ---
scheduler = CosineAnnealingWarmRestarts(trainer.optimizer, T_0=2, T_mult=2)

original_train = trainer.train
def modified_train(*args, **kwargs):
    result = original_train(*args, **kwargs)
    scheduler.step(trainer.state.epoch)
    return result
trainer.train = modified_train

# --- Part D: Train the Model and Finalize ---
print("\n--- Starting Training ---")
trainer.train()
print("--- Training Finished ---")

# === Diagnostics: Model Head and Optimizer State ===
print("\n--- Final Diagnostics ---")
print("Classifier head weights (after train):", model.classifier.weight)
print("Attention pooling weights (after train):", model.vit.pooler.attention_net[0].weight)

opt_params = set([p for group in trainer.optimizer.param_groups for p in group['params']])
print("Classifier weight in optimizer param group:", model.classifier.weight in opt_params)
print("Attention pooling weight in optimizer param group:", model.vit.pooler.attention_net[0].weight in opt_params)

assert model.classifier.weight in opt_params, "CRITICAL ERROR: Classifier weight NOT in optimizer!"
assert model.vit.pooler.attention_net[0].weight in opt_params, "CRITICAL ERROR: Attention pooling weight NOT in optimizer!"

# Save the final model and processor
save_model_and_processor(model, processor, SAVE_DIR)
print(f"--- Model saved to {SAVE_DIR} ---")

--- Setting up optimizer for ViT with Attention Pooling ---
Unfreezing for fine-tuning: 0.attention.attention.query.weight
Unfreezing for fine-tuning: 0.attention.attention.query.bias
Unfreezing for fine-tuning: 0.attention.attention.key.weight
Unfreezing for fine-tuning: 0.attention.attention.key.bias
Unfreezing for fine-tuning: 0.attention.attention.value.weight
Unfreezing for fine-tuning: 0.attention.attention.value.bias
Unfreezing for fine-tuning: 0.attention.output.dense.weight
Unfreezing for fine-tuning: 0.attention.output.dense.bias
Unfreezing for fine-tuning: 0.intermediate.dense.weight
Unfreezing for fine-tuning: 0.intermediate.dense.bias
Unfreezing for fine-tuning: 0.output.dense.weight
Unfreezing for fine-tuning: 0.output.dense.bias
Unfreezing for fine-tuning: 0.layernorm_before.weight
Unfreezing for fine-tuning: 0.layernorm_before.bias
Unfreezing for fine-tuning: 0.layernorm_after.weight
Unfreezing for fine-tuning: 0.layernorm_after.bias
Unfreezing for fine-tuning: 1.attent

/Users/natalyagrokh/miniconda3/envs/ml_expressions/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy
1,0.562600,0.420294,0.969143
2,0.449900,0.417835,0.968857
3,0.445200,0.416936,0.969714
4,0.442000,0.417032,0.970000



Classification Report:
              precision    recall  f1-score   support

       anger       0.98      0.98      0.98       464
     disgust       0.95      0.91      0.93        65
        fear       0.91      0.93      0.92       275
   happiness       1.00      1.00      1.00       615
     neutral       0.98      0.98      0.98       638
 questioning       0.97      0.93      0.95       386
     sadness       0.98      0.97      0.97       316
    surprise       0.97      0.98      0.97       593
    contempt       0.73      0.77      0.75        71
     unknown       0.99      1.00      0.99        77

    accuracy                           0.97      3500
   macro avg       0.95      0.95      0.95      3500
weighted avg       0.97      0.97      0.97      3500


Top 3 confused class pairs:
  - questioning → contempt: 14 instances
  - contempt → questioning: 12 instances
  - surprise → fear: 11 instances

🧠 Avg prediction entropy: 0.3159

🔍 Class entropies (sorted):
  - conte

/Users/natalyagrokh/miniconda3/envs/ml_expressions/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)



Classification Report:
              precision    recall  f1-score   support

       anger       0.99      0.98      0.98       464
     disgust       0.97      0.92      0.94        65
        fear       0.91      0.93      0.92       275
   happiness       1.00      1.00      1.00       615
     neutral       0.98      0.98      0.98       638
 questioning       0.96      0.93      0.95       386
     sadness       0.98      0.97      0.98       316
    surprise       0.97      0.98      0.97       593
    contempt       0.69      0.77      0.73        71
     unknown       0.99      1.00      0.99        77

    accuracy                           0.97      3500
   macro avg       0.94      0.95      0.95      3500
weighted avg       0.97      0.97      0.97      3500


Top 3 confused class pairs:
  - questioning → contempt: 17 instances
  - contempt → questioning: 12 instances
  - surprise → fear: 11 instances

🧠 Avg prediction entropy: 0.3148

🔍 Class entropies (sorted):
  - conte

/Users/natalyagrokh/miniconda3/envs/ml_expressions/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)



Classification Report:
              precision    recall  f1-score   support

       anger       0.99      0.98      0.98       464
     disgust       0.95      0.95      0.95        65
        fear       0.91      0.93      0.92       275
   happiness       1.00      1.00      1.00       615
     neutral       0.98      0.98      0.98       638
 questioning       0.97      0.92      0.95       386
     sadness       0.98      0.97      0.98       316
    surprise       0.97      0.98      0.97       593
    contempt       0.70      0.82      0.75        71
     unknown       0.99      1.00      0.99        77

    accuracy                           0.97      3500
   macro avg       0.94      0.95      0.95      3500
weighted avg       0.97      0.97      0.97      3500


Top 3 confused class pairs:
  - questioning → contempt: 19 instances
  - surprise → fear: 11 instances
  - contempt → questioning: 10 instances

🧠 Avg prediction entropy: 0.3134

🔍 Class entropies (sorted):
  - conte

/Users/natalyagrokh/miniconda3/envs/ml_expressions/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)



Classification Report:
              precision    recall  f1-score   support

       anger       0.99      0.98      0.98       464
     disgust       0.95      0.95      0.95        65
        fear       0.91      0.94      0.92       275
   happiness       1.00      1.00      1.00       615
     neutral       0.99      0.98      0.98       638
 questioning       0.98      0.92      0.95       386
     sadness       0.98      0.97      0.98       316
    surprise       0.97      0.98      0.98       593
    contempt       0.69      0.83      0.76        71
     unknown       0.99      1.00      0.99        77

    accuracy                           0.97      3500
   macro avg       0.94      0.96      0.95      3500
weighted avg       0.97      0.97      0.97      3500


Top 3 confused class pairs:
  - questioning → contempt: 20 instances
  - surprise → fear: 11 instances
  - fear → surprise: 9 instances

🧠 Avg prediction entropy: 0.3220

🔍 Class entropies (sorted):
  - questioning: 

In [19]:
# # --------------------------
# # 16. Rescue & Save from Last Checkpoint (after training)
# # --------------------------
# #in case model save fails, resume from latest checkpoint
# processor.save_pretrained(SAVE_DIR)
# print("✅ Processor manually re-saved.")

# # Use parent directory of SAVE_DIR to locate latest V* folder
# parent_dir = os.path.dirname(SAVE_DIR)
# v_folders = [
#     d for d in os.listdir(parent_dir)
#     if os.path.isdir(os.path.join(parent_dir, d)) and d.startswith("V")
# ]

# def extract_timestamp(name):
#     try:
#         _, date_str, time_str = name.split("_")
#         return datetime.strptime(f"{date_str}_{time_str}", "%Y%m%d_%H%M%S")
#     except Exception:
#         return datetime.min

# latest_version_folder = max(v_folders, key=extract_timestamp)
# latest_version_path = os.path.join(parent_dir, latest_version_folder)
# print(f"🗂️ Using latest version folder: {latest_version_path}")

# # Locate latest checkpoint within that version folder
# checkpoint_dirs = [
#     os.path.join(latest_version_path, d)
#     for d in os.listdir(latest_version_path)
#     if d.startswith("checkpoint-") and os.path.isdir(os.path.join(latest_version_path, d))
# ]
# if not checkpoint_dirs:
#     raise ValueError("❌ No checkpoint found in latest version folder.")

# latest_checkpoint = max(checkpoint_dirs, key=os.path.getmtime)
# print(f"✅ Found latest checkpoint: {latest_checkpoint}")

# # Load model and processor from latest checkpoint and save them
# model = AutoModelForImageClassification.from_pretrained(latest_checkpoint)
# processor = AutoImageProcessor.from_pretrained(latest_version_path)
# model = model.to("cpu")

In [20]:
# --------------------------
# 17. Inference Utilities
# --------------------------

# Reload Model for Inference
model = AutoModelForImageClassification.from_pretrained(SAVE_DIR).to(device).eval()
print("✅ Model reloaded for inference.")

# Single image prediction (unbatched)
def predict_label(image_path, threshold=0.85):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(image, return_tensors="pt").to(device)
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = F.softmax(logits, dim=-1)
        conf, pred_idx = torch.max(probs, dim=-1)
    return (id2label[pred_idx.item()], conf.item()) if conf.item() >= threshold else ("REVIEW", conf.item())

# Batched prediction
def batch_predict(image_folder, batch_size=64, threshold=0.85):
    all_preds = []
    error_count = 0
    image_paths = [
        p for p in Path(image_folder).rglob("*")
        if is_valid_image(p.name)
    ]

    for i in tqdm(range(0, len(image_paths), batch_size), desc="Running inference in batches"):
        batch_paths = image_paths[i:i + batch_size]
        images, valid_paths = [], []

        for path in batch_paths:
            try:
                img = Image.open(path).convert("RGB")
                images.append(img)
                valid_paths.append(str(path))
            except Exception:
                error_count += 1
                continue

        if not images:
            continue

        inputs = processor(images=images, return_tensors="pt").to(device)
        with torch.no_grad():
            logits = model(**inputs).logits
            probs = F.softmax(logits, dim=-1)
            confs, preds = torch.max(probs, dim=-1)

        for pred, conf, path in zip(preds.tolist(), confs.tolist(), valid_paths):
            all_preds.append(LABEL_NAMES[pred] if conf >= threshold else "REVIEW")

    print(f"✅ Inference complete. Skipped {error_count} invalid image(s).")
    return all_preds

# Distribution plot
def plot_distribution(predictions, output_path):
    label_counts = Counter(predictions)
    labels = sorted(label_counts.keys())
    counts = [label_counts[label] for label in labels]

    plt.figure(figsize=(10, 5))
    plt.bar(labels, counts)
    plt.title("Predicted Expression Distribution")
    plt.xlabel("Expression")
    plt.ylabel("Count")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()

Some weights of the model checkpoint at /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V21_20250630_142520 were not used when initializing ViTForImageClassification: ['vit.pooler.attention_net.0.bias', 'vit.pooler.attention_net.0.weight', 'vit.pooler.attention_net.2.bias', 'vit.pooler.attention_net.2.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


✅ Model reloaded for inference.


In [21]:
# --------------------------
# 18. Entry Point for Inference
# --------------------------
if __name__ == "__main__" and RUN_INFERENCE:

    # Auto-locate latest model directory
    OUTPUT_PATH = os.path.join(SAVE_DIR, f"{VERSION}_distribution_plot_{timestamp}.png")

    predictions = batch_predict(IMAGE_DIR)
    reviewed_paths = []
    image_paths = [str(p) for p in Path(IMAGE_DIR).rglob("*") if is_valid_image(p.name)]

    for path, label in zip(image_paths, predictions):
        if label == "REVIEW":
            reviewed_paths.append(path)

    # Save paths to inspect manually
    with open(os.path.join(SAVE_DIR, f"{VERSION}_review_candidates.txt"), "w") as f:
        f.write("\n".join(reviewed_paths))
    print(f"📝 Saved REVIEW file paths to {VERSION}_review_candidates.txt")

    plot_distribution(predictions, OUTPUT_PATH)
    print(f"Distribution plot saved to: {OUTPUT_PATH}")

Running inference in batches: 100%|███████████| 274/274 [15:11<00:00,  3.33s/it]

✅ Inference complete. Skipped 0 invalid image(s).
📝 Saved REVIEW file paths to V21_review_candidates.txt
Distribution plot saved to: /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V21_20250630_142520/V21_distribution_plot_20250630_142520.png


In [22]:
# --------------------------
# 19. Temperature Scaling Calibration 
# --------------------------

# Wrapper model for calibrated inference
class ModelWithTemperature(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.temperature = nn.Parameter(torch.ones(1) * 1.5)

    def forward(self, input_ids=None, pixel_values=None, **kwargs):
        logits = self.model(pixel_values=pixel_values).logits
        return logits / self.temperature

    def set_temperature(self, logits, labels):
        nll_criterion = nn.CrossEntropyLoss()
        optimizer = LBFGS([self.temperature], lr=0.01, max_iter=50)

        def eval_fn():
            optimizer.zero_grad()
            loss = nll_criterion(logits / self.temperature, labels)
            loss.backward()
            return loss

        optimizer.step(eval_fn)
        print(f"Optimal temperature (wrapped): {self.temperature.item():.4f}")
        return self

# Dynamically locate the most recent V* folder that contains logits/labels
base_dir = os.path.dirname(SAVE_DIR)
v_folders = sorted([
    d for d in os.listdir(base_dir)
    if os.path.isdir(os.path.join(base_dir, d)) and d.startswith("V")
], key=lambda d: os.path.getmtime(os.path.join(base_dir, d)), reverse=True)

logits_path, labels_path = None, None
for v in v_folders:
    version_tag = v.split('_')[0]
    folder_path = os.path.join(base_dir, v)
    logits_candidate = os.path.join(folder_path, f"logits_eval_{version_tag}.npy")
    labels_candidate = os.path.join(folder_path, f"labels_eval_{version_tag}.npy")
    if os.path.exists(logits_candidate) and os.path.exists(labels_candidate):
        INFER_SAVE_DIR = folder_path
        INFER_VERSION = version_tag
        print(f"📁 Using calibration files from: {SAVE_DIR}")
        logits_path = logits_candidate
        labels_path = labels_candidate
        break

# --------------------------
# Run calibration
# --------------------------
if logits_path and labels_path:
    result = apply_temperature_scaling(logits_path, labels_path)
    if result is not None:
        temperature, logits, labels = result
        plot_reliability_diagram(logits, labels, temperature)
else:
    print(f"⚠️ Skipping temperature scaling and diagram (missing logits or labels in {SAVE_DIR})")

📁 Using calibration files from: /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V21_20250630_142520
📂 Loading logits from: /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V21_20250630_142520/logits_eval_V21.npy
📂 Loading labels from: /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V21_20250630_142520/labels_eval_V21.npy
✅ Optimal temperature: 1.1300
✅ Calibrated Log Loss: 0.1740
📊 Saved reliability diagram to /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V21_20250630_142520/V21_reliability_diagram_calibrated.png


In [23]:
# --------------------------
# 20. Review & Relabel 'REVIEW' Predictions (with Audit Logging & Clustering)
# --------------------------
MINORITY_LABELS = ["disgust", "contempt", "fear", "questioning"]
MINORITY_ENTROPY_THRESH = 0.6
REVIEW_THRESHOLD = 0.85

REVIEW_BY_CLASS_DIR = os.path.join(SAVE_DIR, "review_predictions_by_class")
REVIEW_CSV_LOG = os.path.join(SAVE_DIR, f"{VERSION}_review_predictions_with_preds.csv")
REVIEW_CLUSTER_DIR = os.path.join(SAVE_DIR, "review_predictions_clustered")

os.makedirs(REVIEW_BY_CLASS_DIR, exist_ok=True)
os.makedirs(REVIEW_CLUSTER_DIR, exist_ok=True)

# ---- If you HAVE NOT already generated review CSV (inference stage) ----
if not os.path.exists(REVIEW_CSV_LOG):
    review_log = []
    image_paths = [
        p for p in Path(IMAGE_DIR).rglob("*")
        if p.is_file() and p.suffix.lower() in [".jpg", ".jpeg", ".png"]
    ]
    
    for img_path in image_paths:
        try:
            image = Image.open(img_path).convert("RGB")
            inputs = processor(image, return_tensors="pt").to(device)
            with torch.no_grad():
                logits = model(**inputs).logits
                probs = F.softmax(logits, dim=-1)
                conf, pred_idx = torch.max(probs, dim=-1)
            conf_val = conf.item()
            pred_label = id2label[pred_idx.item()]
            
            entropy = -torch.sum(probs * torch.log(probs + 1e-12), dim=-1).item()

            if pred_label in MINORITY_LABELS and entropy > MINORITY_ENTROPY_THRESH:
                tag = "unknown"
            elif conf_val < REVIEW_THRESHOLD:
                tag = "REVIEW"
            else:
                tag = pred_label
            
            review_log.append({
                "image_path": str(img_path),
                "predicted_label": pred_label,
                "confidence": round(conf_val, 4),
                "entropy": round(entropy, 4),
                "tag": tag
            })
            
            # For backward compatibility, still copy to REVIEW_BY_CLASS_DIR if tag is not "unknown"
            if tag not in ["unknown"]:
                target_dir = os.path.join(REVIEW_BY_CLASS_DIR, tag)
                os.makedirs(target_dir, exist_ok=True)
                shutil.copy(str(img_path), target_dir)
        except Exception as e:
            print(f"⚠️ Error with image: {img_path} | {e}")
            
    pd.DataFrame(review_log).to_csv(REVIEW_CSV_LOG, index=False)
    print(f"✅ Completed tagging + copying REVIEW predictions to: {REVIEW_BY_CLASS_DIR}")
    print(f"📄 CSV log saved to: {REVIEW_CSV_LOG}")

# ---- If you already HAVE a review CSV (assignment/audit stage) ----
df = pd.read_csv(REVIEW_CSV_LOG)
review_assignment_log = []
for _, row in df.iterrows():
    path = row["image_path"]
    pred_label = row["predicted_label"]
    conf = float(row["confidence"])
    true_label = os.path.basename(os.path.dirname(path))
    
    entropy = float(row.get("entropy", 0))  # default to 0 if not present
    if pred_label in MINORITY_LABELS and entropy > MINORITY_ENTROPY_THRESH:
        assigned = "unknown"
    elif conf < REVIEW_THRESHOLD:
        assigned = "REVIEW"
    else:
        assigned = pred_label
    
    dest_dir = os.path.join(REVIEW_BY_CLASS_DIR, assigned)
    os.makedirs(dest_dir, exist_ok=True)
    shutil.copy(path, dest_dir)
    review_assignment_log.append([path, true_label, pred_label, conf, assigned, entropy])

log_df = pd.DataFrame(
    review_assignment_log,
    columns=["image_path", "true_label", "pred_label", "confidence", "assigned_folder", "entropy"]
)

log_df.to_csv(os.path.join(SAVE_DIR, "review_assignment_audit.csv"), index=False)
print("✅ Review assignments (with audit) complete.")

print("Assignment summary:", Counter(log_df["assigned_folder"]))

# ---- Perceptual hash clustering of review pool ----
def phash_distance(hash1, hash2):
    return hash1 - hash2

PHASH_CLUSTER_THRESHOLD = 6
image_paths = [row[0] for row in review_assignment_log if row[4] != "unknown"]  # assigned to a class

hashes = []
for img_path in image_paths:
    try:
        img = Image.open(img_path).convert("L").resize((64, 64))
        hashes.append(hex_to_hash(str(phash(img))))
    except Exception as e:
        print(f"phash error: {img_path} | {e}")

clusters = []
used = set()
for i, h1 in enumerate(hashes):
    if i in used:
        continue
    cluster = [image_paths[i]]
    used.add(i)
    for j, h2 in enumerate(hashes):
        if j <= i or j in used:
            continue
        if phash_distance(h1, h2) <= PHASH_CLUSTER_THRESHOLD:
            cluster.append(image_paths[j])
            used.add(j)
    if len(cluster) > 1:
        clusters.append(cluster)

if not clusters:
    print(f"⚠️ No clusters found for review. {REVIEW_CLUSTER_DIR} will remain empty.")
else:
    for idx, cluster in enumerate(clusters):
        out_dir = os.path.join(REVIEW_CLUSTER_DIR, f"cluster_{idx}")
        os.makedirs(out_dir, exist_ok=True)
        for p in cluster:
            shutil.copy(p, out_dir)
    print(f"✅ Saved {len(clusters)} clusters to {REVIEW_CLUSTER_DIR}")

✅ Completed tagging + copying REVIEW predictions to: /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V21_20250630_142520/review_predictions_by_class
📄 CSV log saved to: /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V21_20250630_142520/V21_review_predictions_with_preds.csv
✅ Review assignments (with audit) complete.
Assignment summary: Counter({'neutral': 3159, 'happiness': 2946, 'surprise': 2890, 'anger': 2335, 'sadness': 1604, 'fear': 1387, 'unknown': 1189, 'questioning': 705, 'REVIEW': 591, 'contempt': 303, 'disgust': 251})
✅ Saved 265 clusters to /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V21_20250630_142520/review_predictions_clustered


In [24]:
# --------------------------
# 21. REVIEW Pool Diagnostics & Hard Confusion Mining
# --------------------------

# A. Flag hard confusion pairs for manual review
REVIEW_CONFUSION_PAIRS = [("contempt", "questioning"), ("fear", "surprise")]

def parse_review_confusions(csv_path, confusion_pairs):
    import csv
    flagged_imgs = {pair: [] for pair in confusion_pairs}
    with open(csv_path) as f:
        reader = csv.DictReader(f)
        for row in reader:
            pred = row["predicted_label"]
            true = os.path.basename(os.path.dirname(row["image_path"]))
            conf = float(row["confidence"])
            for a, b in confusion_pairs:
                if ((pred == a and true == b) or (pred == b and true == a)) and conf < 0.8:
                    flagged_imgs[(a, b)].append(row["image_path"])
    return flagged_imgs

confusion_candidates = parse_review_confusions(REVIEW_CSV_LOG, REVIEW_CONFUSION_PAIRS)
for pair, imgs in confusion_candidates.items():
    print(f"\nFlagged {len(imgs)} hard negatives for {pair}:")
    out_path = os.path.join(SAVE_DIR, f"review_hardneg_{pair[0]}_{pair[1]}.txt")
    with open(out_path, "w") as f:
        f.write("\n".join(imgs))
    print(f"  Saved list: {out_path}")

# B. Organize REVIEW-tagged images by predicted class (for curation)
REVIEW_SORT_DIR = os.path.join(SAVE_DIR, "review_predictions_by_class")
os.makedirs(REVIEW_SORT_DIR, exist_ok=True)
review_txt_path = os.path.join(SAVE_DIR, f"{VERSION}_review_candidates.txt")
csv_path = os.path.join(SAVE_DIR, f"{VERSION}_review_predictions_with_preds.csv")

if os.path.exists(review_txt_path) and os.path.exists(csv_path):
    with open(review_txt_path, "r") as f:
        review_paths = {line.strip() for line in f.readlines()}

    df = pd.read_csv(csv_path)
    count = 0

    print(f"🔍 Found {len(df)} total predictions (CSV) and {len(review_paths)} REVIEW-tagged paths.")

    for _, row in df.iterrows():
        path = row["image_path"]
        label = row["predicted_label"]
        conf = row["confidence"]

        if path in review_paths and label != "REVIEW":
            dest_dir = os.path.join(REVIEW_SORT_DIR, label)
            os.makedirs(dest_dir, exist_ok=True)
            shutil.copy(path, dest_dir)
            count += 1

    print(f"📂 Grouped {count} REVIEW images into folders by predicted label in: {REVIEW_SORT_DIR}")
else:
    print("⚠️ Missing review candidates file or prediction CSV.")


Flagged 7 hard negatives for ('contempt', 'questioning'):
  Saved list: /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V21_20250630_142520/review_hardneg_contempt_questioning.txt

Flagged 10 hard negatives for ('fear', 'surprise'):
  Saved list: /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V21_20250630_142520/review_hardneg_fear_surprise.txt
🔍 Found 17360 total predictions (CSV) and 1326 REVIEW-tagged paths.
📂 Grouped 1315 REVIEW images into folders by predicted label in: /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V21_20250630_142520/review_predictions_by_class


In [25]:
# --------------------------
# 22. Visualization & Error Tracking
# --------------------------

print("Label name/id mapping:")
for idx, name in enumerate(LABEL_NAMES):
    print(f"{idx}: {name}")

# Defensive: Check that metrics file exists before plotting
per_class_csv = os.path.join(SAVE_DIR, "per_class_metrics.csv")
if not os.path.exists(per_class_csv):
    print(f"⚠️ Metrics file {per_class_csv} not found.")
else:
    metrics_df = pd.read_csv(per_class_csv)
    last_row = metrics_df.iloc[-1]

    fig, ax = plt.subplots(figsize=(10,6))
    f1s = [last_row[f"f1_{n}"] for n in LABEL_NAMES]
    ax.bar(LABEL_NAMES, f1s)
    ax.set_title("Per-Class F1 (Last Epoch)")
    plt.tight_layout()
    plt.savefig(os.path.join(SAVE_DIR, "per_class_f1.png"))
    plt.close()

    # Bar plot of per-class entropy
    entropies = [last_row[f"entropy_{n}"] for n in LABEL_NAMES]
    fig, ax = plt.subplots(figsize=(10,6))
    ax.bar(LABEL_NAMES, entropies)
    ax.set_title("Per-Class Mean Entropy (Last Epoch)")
    plt.tight_layout()
    plt.savefig(os.path.join(SAVE_DIR, "per_class_entropy.png"))
    plt.close()

    # Histogram for REVIEW pool
    review_counts = Counter()
    if os.path.exists(REVIEW_SORT_DIR):
        for label_dir in os.listdir(REVIEW_SORT_DIR):
            count = len(os.listdir(os.path.join(REVIEW_SORT_DIR, label_dir)))
            review_counts[label_dir] = count
        plt.bar(review_counts.keys(), review_counts.values())
        plt.title("REVIEW Pool Distribution by Predicted Class")
        plt.savefig(os.path.join(SAVE_DIR, "review_pool_distribution.png"))
        plt.close()
        # Flag if >70% in one class
        total = sum(review_counts.values())
        for label, count in review_counts.items():
            if total > 0 and count / total > 0.7:
                print(f"⚠️ REVIEW pool highly imbalanced: {count/total:.1%} in '{label}'")

    # Audit print block (as before)
    print("Sample review predictions (audit):")
    if 'log_df' in locals():
        print(log_df[["image_path", "true_label", "pred_label", "confidence", "assigned_folder"]].head())
    elif 'df' in locals():
        print(df[["image_path", "true_label", "predicted_label", "confidence"]].head())
    else:
        print("No review/audit DataFrame found for printing.")

# ✅ AUDIT BLOCK
print("Sample review predictions (audit):")
if 'log_df' in locals():
    print(log_df[["image_path", "true_label", "pred_label", "confidence", "assigned_folder"]].head())
elif 'df' in locals():
    print(df[["image_path", "true_label", "predicted_label", "confidence"]].head())
else:
    print("No review/audit DataFrame found for printing.")

Label name/id mapping:
0: anger
1: disgust
2: fear
3: happiness
4: neutral
5: questioning
6: sadness
7: surprise
8: contempt
9: unknown
Sample review predictions (audit):
                                          image_path true_label pred_label  \
0  /Users/natalyagrokh/AI/ml_expressions/img_data...   contempt   contempt   
1  /Users/natalyagrokh/AI/ml_expressions/img_data...   contempt   contempt   
2  /Users/natalyagrokh/AI/ml_expressions/img_data...   contempt    neutral   
3  /Users/natalyagrokh/AI/ml_expressions/img_data...   contempt   contempt   
4  /Users/natalyagrokh/AI/ml_expressions/img_data...   contempt   contempt   

   confidence assigned_folder  
0      0.9175        contempt  
1      0.9405        contempt  
2      0.9376         neutral  
3      0.9468        contempt  
4      0.9372        contempt  
Sample review predictions (audit):
                                          image_path true_label pred_label  \
0  /Users/natalyagrokh/AI/ml_expressions/img_data...   

In [26]:
# --------------------------
# 23. Deployment Readiness Assertions and Flags
# --------------------------

# Load metrics
metrics_df = pd.read_csv(os.path.join(SAVE_DIR, "per_class_metrics.csv"))
last = metrics_df.iloc[-1]
warn = False

for cname in LABEL_NAMES:
    f1 = last[f"f1_{cname}"]
    entropy = last[f"entropy_{cname}"]
    if f1 < 0.8:
        print(f"🚨 F1 < 0.8 for class '{cname}': {f1:.2f}")
        warn = True
    if entropy > 0.4:
        print(f"🚨 Entropy > 0.4 for class '{cname}': {entropy:.2f}")
        warn = True

if not warn:
    print("✅ All classes ready for deployment: F1 >= 0.8 and entropy <= 0.4")
else:
    print("⚠️ Some classes not deployment-ready! Address above issues before production.")


🚨 Entropy > 0.4 for class 'questioning': 0.50
🚨 F1 < 0.8 for class 'contempt': 0.76
🚨 Entropy > 0.4 for class 'contempt': 0.49
⚠️ Some classes not deployment-ready! Address above issues before production.


In [30]:
# ===================================================================
# 24. Model Ensembling Analysis
# ===================================================================

print("--- Setting up Model Ensembling Analysis ---")

# --- Part A: Dynamic Model Loading ---
# This section dynamically finds the two most recent, successfully saved model versions.

MODEL_ROOT = "/Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training"

def extract_version_from_path(path):
    match = re.search(r"V(\d+)", os.path.basename(path))
    return int(match.group(1)) if match else -1

# Find all valid model directories that contain saved model weights
all_model_dirs = [
    os.path.join(MODEL_ROOT, d)
    for d in os.listdir(MODEL_ROOT)
    if d.startswith("V") and os.path.isdir(os.path.join(MODEL_ROOT, d))
       and (os.path.exists(os.path.join(MODEL_ROOT, d, "model.safetensors")) or 
            os.path.exists(os.path.join(MODEL_ROOT, d, "pytorch_model.bin")))
]

# Exclude the current run's directory
all_model_dirs = [d for d in all_model_dirs if os.path.abspath(d) != os.path.abspath(SAVE_DIR)]

# Sort by version number and select the two most recent
if len(all_model_dirs) < 2:
    print("⚠️ Found fewer than two previous model versions. Skipping ensembling.")
else:
    sorted_models = sorted(all_model_dirs, key=extract_version_from_path, reverse=True)
    model_path_1 = sorted_models[0] # Most recent (e.g., V20)
    model_path_2 = sorted_models[1] # Second most recent (e.g., V19)

    print(f"✅ Dynamically selected models for ensembling:")
    print(f"   - Model 1: {os.path.basename(model_path_1)}")
    print(f"   - Model 2: {os.path.basename(model_path_2)}")

    # Load the models
    model_1 = AutoModelForImageClassification.from_pretrained(model_path_1).to(device).eval()
    model_2 = AutoModelForImageClassification.from_pretrained(model_path_2).to(device).eval()
    ensemble_models = [model_1, model_2]


    # Returns a boolean tensor: True if the prediction is low-confidence
    def is_uncertain(probs, threshold=0.85, entropy_thresh=1.3):
        conf, _ = torch.max(probs, dim=-1)
        entropy = -torch.sum(probs * torch.log(probs + 1e-12), dim=-1)
        return (conf < threshold) | (entropy > entropy_thresh)
    
        # --- Part B: Ensemble Prediction Function ---
        def ensemble_predict(models, processor, image_path, device="cpu"):
            image = Image.open(image_path).convert("RGB")
            inputs = processor(image, return_tensors="pt").to(device)
            softmaxes = []
            individual_preds = []
    
            for m in models:
                with torch.no_grad():
                    logits = m(**inputs).logits
                    probs = F.softmax(logits, dim=-1)
                    individual_preds.append(id2label[torch.argmax(probs, dim=-1).item()])
                    softmaxes.append(probs.cpu().numpy())
            
            avg_probs = np.mean(softmaxes, axis=0)
            ensemble_pred_idx = np.argmax(avg_probs)
            ensemble_conf = avg_probs[0, ensemble_pred_idx]
            
            return id2label[ensemble_pred_idx], ensemble_conf, individual_preds
    

    # --- Part C: Run Analysis on Misclassified Images ---
    # Load the audit file from the most recent run to find challenging images
    audit_csv_path = os.path.join(SAVE_DIR, "review_assignment_audit.csv")
    if os.path.exists(audit_csv_path):
        print("\n--- Running Ensemble Analysis on Hard Cases ---")
        audit_df = pd.read_csv(audit_csv_path)
        
        # Filter for images that were misclassified by the single model
        misclassified_df = audit_df[audit_df['true_label'] != audit_df['pred_label']]
        
        # Analyze the top 5 hardest cases
        for _, row in misclassified_df.head(5).iterrows():
            image_path = row['image_path']
            true_label = row['true_label']
            single_model_pred = row['pred_label']
            
            ensemble_pred, ensemble_conf, individual_preds = ensemble_predict(
                ensemble_models, processor, image_path, device=device
            )
            
            print(f"\nImage: {os.path.basename(image_path)}")
            print(f"  - True Label:      {true_label}")
            print(f"  - Model 1 ({os.path.basename(model_path_1)}) Pred: {individual_preds[0]}")
            print(f"  - Model 2 ({os.path.basename(model_path_2)}) Pred: {individual_preds[1]}")
            print(f"  - ENSEMBLE Pred:   {ensemble_pred} (Confidence: {ensemble_conf:.2f})")
            
            if ensemble_pred == true_label and single_model_pred != true_label:
                print("  - ✅ SUCCESS: Ensemble corrected the misclassification!")
            elif ensemble_pred != true_label:
                print("  - ❌ FAILURE: Ensemble also failed to classify correctly.")
            else:
                print("  - ⚠️ NOTE: Ensemble agreed with a correct single-model prediction (edge case).")
    else:
        print(f"⚠️ Could not find audit CSV at {audit_csv_path}. Skipping ensemble analysis.")

--- Setting up Model Ensembling Analysis ---
✅ Dynamically selected models for ensembling:
   - Model 1: V20_20250629_134956
   - Model 2: V19_20250628_131955

--- Running Ensemble Analysis on Hard Cases ---

Image: img_2617.jpg
  - True Label:      contempt
  - Model 1 (V20_20250629_134956) Pred: neutral
  - Model 2 (V19_20250628_131955) Pred: neutral
  - ENSEMBLE Pred:   neutral (Confidence: 0.95)
  - ❌ FAILURE: Ensemble also failed to classify correctly.

Image: Iain_Richmond_0001.jpg
  - True Label:      contempt
  - Model 1 (V20_20250629_134956) Pred: neutral
  - Model 2 (V19_20250628_131955) Pred: neutral
  - ENSEMBLE Pred:   neutral (Confidence: 0.96)
  - ❌ FAILURE: Ensemble also failed to classify correctly.

Image: Richard_Gephardt_0001.jpg
  - True Label:      contempt
  - Model 1 (V20_20250629_134956) Pred: happiness
  - Model 2 (V19_20250628_131955) Pred: happiness
  - ENSEMBLE Pred:   happiness (Confidence: 0.92)
  - ❌ FAILURE: Ensemble also failed to classify correctly.

